In [1]:
#참고 블로그*
#https://banana-media-lab.tistory.com/entry/Librosa-python-library%EB%A1%9C-%EC%9D%8C%EC%84%B1%ED%8C%8C%EC%9D%BC-%EB%B6%84%EC%84%9D%ED%95%98%EA%B8%B0


#1. 음악 데이터 파일 로드에 필요한 라이브러리 
import IPython.display
import pandas as pd
import numpy as np

# 음악 분석 라이브러리
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [2]:
#2. 음악 정보 파일 불러오기
# music_inform.csv 

music_inform=pd.read_csv('music_inform.csv')

display(music_inform)

,Music,Genre,ID,Format,Path
0,Contact High - RKVC,RnBSoul_Calm,R_C1,mp3,RnBSoul_Calm/R_C1.mp3
1,QuangerineCream - Noir Et Blanc Vie,RnBSoul_Calm,R_C2,mp3,RnBSoul_Calm/R_C2.mp3
2,TrueArtRealAffectionPart4 - Noir Et Blanc Vie,RnBSoul_Calm,R_C3,mp3,RnBSoul_Calm/R_C3.mp3
3,Wallflowers - Bad Snacks,RnBSoul_Calm,R_C4,mp3,RnBSoul_Calm/R_C4.mp3
4,Where To_ - Birocratic,RnBSoul_Calm,R_C5,mp3,RnBSoul_Calm/R_C5.mp3
5,Goji Berry Beat - Nana Kwabena,Hiphop_Funky,H_F1,mp3,Hiphop_Funky/H_F1.mp3
6,Madam Wahala Beat - Nana Kwabena,Hiphop_Funky,H_F2,mp3,Hiphop_Funky/H_F2.mp3
7,Pretty Boy - DJ Freedem,Hiphop_Funky,H_F3,mp3,Hiphop_Funky/H_F3.mp3
8,Road Tripzzz - Ofshane,Hiphop_Funky,H_F4,mp3,Hiphop_Funky/H_F4.mp3
9,Twisted Bandits All Around Me - NoMBe,Hiphop_Funky,H_F5,mp3,Hiphop_Funky/H_F5.mp3


In [3]:
#3. 분석에 필요한 열만 추출 : pitch_data
#ID, Path

music_data=music_inform[['ID','Path']]

display(music_data)

,ID,Path
0,R_C1,RnBSoul_Calm/R_C1.mp3
1,R_C2,RnBSoul_Calm/R_C2.mp3
2,R_C3,RnBSoul_Calm/R_C3.mp3
3,R_C4,RnBSoul_Calm/R_C4.mp3
4,R_C5,RnBSoul_Calm/R_C5.mp3
5,H_F1,Hiphop_Funky/H_F1.mp3
6,H_F2,Hiphop_Funky/H_F2.mp3
7,H_F3,Hiphop_Funky/H_F3.mp3
8,H_F4,Hiphop_Funky/H_F4.mp3
9,H_F5,Hiphop_Funky/H_F5.mp3


In [4]:
#4. 모든 곡들의 mfcc 저장 
mfcc_info=[]

for i in range(len(music_data)):
    music_name=music_data.loc[i]
    music_path=music_name[1]
    
    y,sr=librosa.load(music_path,sr=44100,duration=10)
    IPython.display.Audio(data=y,rate=sr)


    #mfcc 추출
    mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=13)
    mfcc_info.append(mfcc.T)

c:\users\yeongeun\appdata\local\programs\python\python38-32\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [5]:
#test
mfcc_info[0][861]

array([-382.05444  ,  169.09964  ,   89.73183  ,   80.49739  ,
         54.637215 ,   18.392488 ,    4.002273 ,    5.517345 ,
         -3.2892992,  -10.993269 ,   -4.1220703,   -5.4973297,
         -6.0789022], dtype=float32)

In [6]:
#5. 유사도 구하기
n=len(music_data) #데이터 수 
frame_num=mfcc_info[0].shape[0] #프레임의 개수

similarity=[[0 for i in range(n)] for j in range(n)]

In [7]:
#5-0. mfcc 13계수의 차이값 구하기
def mfcc_difference(mfcc1,mfcc2):
    difference=0
    
    for i in range(13):
        difference+=abs(mfcc1[i]-mfcc2[i])
        
    return difference
    

In [8]:
#i곡과 j곡의 유사 구하기
for i in range(n):
    for j in range(i,n):
        
        #동일 곡과의 유사도는 구하지 않음 
        if i==j:
            continue 
            
        #5-1. dtw를 구하기 위한 행렬
        dtw=[[0 for i in range(frame_num)] for j in range(frame_num)]
        
        #5-2. dtw 첫행값 구하기
        dtw[0][0]=mfcc_difference(mfcc_info[i][0],mfcc_info[j][0])
        
        for dj in range(1,frame_num):
            dtw[0][dj]=dtw[0][dj-1]+mfcc_difference(mfcc_info[i][0],mfcc_info[j][dj])
        
        #5-3. dtw 첫열값 구하기
        for di in range(1,frame_num):
            dtw[di][0]=dtw[di-1][0]+mfcc_difference(mfcc_info[i][di],mfcc_info[j][0])
            
        #5-4. dtw 채우기
        for di in range(1,frame_num):
            for dj in range(1,frame_num):
                min_dtw=min(min(dtw[di-1][dj-1],dtw[di-1][dj]),dtw[di][dj-1])
                
                dtw[di][dj]=min_dtw+mfcc_difference(mfcc_info[i][di],mfcc_info[j][dj])
                
        #5-5. 최종 dtw 유사거리 저장
        similarity[i][j]=int(dtw[frame_num-1][frame_num-1])
        similarity[j][i]=int(dtw[frame_num-1][frame_num-1])
        
        #현재 진행상황 파악하기 위한 print
        print('[',i,']','[',j,']의 유사도 : ', similarity[i][j])
        
        

[ 0 ] [ 1 ]의 유사도 :  360547
[ 0 ] [ 2 ]의 유사도 :  403032
[ 0 ] [ 3 ]의 유사도 :  283880
[ 0 ] [ 4 ]의 유사도 :  403200
[ 0 ] [ 5 ]의 유사도 :  350400
[ 0 ] [ 6 ]의 유사도 :  364448
[ 0 ] [ 7 ]의 유사도 :  279424
[ 0 ] [ 8 ]의 유사도 :  365447
[ 0 ] [ 9 ]의 유사도 :  376268
[ 0 ] [ 10 ]의 유사도 :  481972
[ 0 ] [ 11 ]의 유사도 :  399549
[ 0 ] [ 12 ]의 유사도 :  303367
[ 0 ] [ 13 ]의 유사도 :  373791
[ 0 ] [ 14 ]의 유사도 :  292412
[ 1 ] [ 2 ]의 유사도 :  282307
[ 1 ] [ 3 ]의 유사도 :  318248
[ 1 ] [ 4 ]의 유사도 :  293020
[ 1 ] [ 5 ]의 유사도 :  290120
[ 1 ] [ 6 ]의 유사도 :  440733
[ 1 ] [ 7 ]의 유사도 :  405401
[ 1 ] [ 8 ]의 유사도 :  454935
[ 1 ] [ 9 ]의 유사도 :  311479
[ 1 ] [ 10 ]의 유사도 :  344723
[ 1 ] [ 11 ]의 유사도 :  332284
[ 1 ] [ 12 ]의 유사도 :  403785
[ 1 ] [ 13 ]의 유사도 :  312518
[ 1 ] [ 14 ]의 유사도 :  355318
[ 2 ] [ 3 ]의 유사도 :  320003
[ 2 ] [ 4 ]의 유사도 :  352765
[ 2 ] [ 5 ]의 유사도 :  279291
[ 2 ] [ 6 ]의 유사도 :  387994
[ 2 ] [ 7 ]의 유사도 :  430407
[ 2 ] [ 8 ]의 유사도 :  565968
[ 2 ] [ 9 ]의 유사도 :  261547
[ 2 ] [ 10 ]의 유사도 :  223130
[ 2 ] [ 11 ]의 유사도 :  403366
[ 2 ] [ 12 ]의 유사

In [9]:
#6. 5곡정도씩 추천 리스트 만들어보기 
#r_num : 추천하고 싶은 곡 수 : 5
#music_num : 자기 자신 번호
#s: similarity[music_num]

def recommendation(music_num,s,r_num):
    s_list=[]
    
    for i in range(len(s)):
        if music_num==i:
            continue
        s_list.append([s[i],i])
        
    s_list=sorted(s_list)
    
    return [s_list[i][1] for i in range(r_num)]


r_num=5
for i in range(n):
    print(i,'곡의 추천 곡 : ',recommendation(i,similarity[i],r_num))
    
    

0 곡의 추천 곡 :  [7, 3, 14, 12, 5]
1 곡의 추천 곡 :  [2, 5, 4, 9, 13]
2 곡의 추천 곡 :  [10, 9, 5, 1, 13]
3 곡의 추천 곡 :  [14, 0, 9, 5, 4]
4 곡의 추천 곡 :  [13, 5, 11, 1, 9]
5 곡의 추천 곡 :  [13, 4, 2, 9, 11]
6 곡의 추천 곡 :  [14, 12, 7, 0, 3]
7 곡의 추천 곡 :  [0, 14, 12, 5, 3]
8 곡의 추천 곡 :  [7, 12, 0, 11, 3]
9 곡의 추천 곡 :  [2, 10, 5, 3, 4]
10 곡의 추천 곡 :  [2, 9, 5, 13, 1]
11 곡의 추천 곡 :  [4, 5, 13, 1, 14]
12 곡의 추천 곡 :  [0, 7, 6, 14, 8]
13 곡의 추천 곡 :  [5, 4, 2, 11, 9]
14 곡의 추천 곡 :  [3, 6, 5, 0, 7]


In [15]:
#7. 곡별 추천 리스트 파일로 저장하기

#recomsize: 추천 리스트
#header 생성
recomsize=['recom'+str(i) for i in range(1,r_num+1)]
header=['music_id']+recomsize

recom_list=[]

for i in range(n):
    recom_list.append([i]+recommendation(i,similarity[i],r_num))
    
recom_df=pd.DataFrame(recom_list,columns=header)
display(recom_df)

#csv파일로 저장
recom_df.to_csv('mfcc_recommendation.csv',header=True,index=False)

,music_id,recom1,recom2,recom3,recom4,recom5
0,0,7,3,14,12,5
1,1,2,5,4,9,13
2,2,10,9,5,1,13
3,3,14,0,9,5,4
4,4,13,5,11,1,9
5,5,13,4,2,9,11
6,6,14,12,7,0,3
7,7,0,14,12,5,3
8,8,7,12,0,11,3
9,9,2,10,5,3,4
